In [1]:


# neuracrawl-tuner
# Import all functionality from the library module

from neuracrawl_tuner_lib import (
    PROJECT_MANAGER,
    save_sitemap_urls,
    extract_frequent_sitemap_urls,
    extract_url_extensions,
    extract_url_regexes,
    extract_interesting_urls,
    download_interesting_urls,
    extract_css_selectors,
    apply_css_selectors,
    discover_urls,
)



In [2]:
# ===== CUSTOMER NAME =====
CUSTOMER = "neuraflow"
# =====================================

# Derived paths
DOMAIN_URL = f"https://www.{CUSTOMER}.de"
LOG_FILE = f"{CUSTOMER}_discovery.log"

PROJECT_MANAGER.set_project(CUSTOMER)

In [ ]:


from nest_asyncio import apply
apply()



result = await discover_urls(
    DOMAIN_URL,
    max_pages=5000,
    url_exclusion_patterns=[
        r"(aktuelles|meldungen|veranstaltungen-details|event|currentPage|dachauer-volksblatt|/dokumente/)",
        r"\.(ics|pdf|json|vcf|kml|gpx)",
        r"\?",
    ],
    crawl_sample=10000,
    log_file_path=LOG_FILE,
)
print(f"\n📊 Total: {result['total']} URLs")

save_sitemap_urls(sorted(result["urls"]))
print(f"✓ Saved to sitemap_urls.txt")
print(f"✓ Log saved to {LOG_FILE}")


## Sitemap

In [3]:
extract_frequent_sitemap_urls(5)
extract_url_extensions()

Found 2 common URL areas with frequency >= 5


## Exclusion URL Regexes

In [4]:
await extract_url_regexes(
    ""
)

INFO:neuracrawl_tuner_lib:Extracting URL regexes...
INFO:neuracrawl_tuner_lib:Analyzing frequent URL paths...
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:neuracrawl_tuner_lib:Found 0 URL regexes.
INFO:neuracrawl_tuner_lib:Extracted 0 URL regexes.
INFO:neuracrawl_tuner_lib:Applying URL regexes to sitemap URLs...
INFO:neuracrawl_tuner_lib:Applied URL regexes. Found 0 excluded and 53 non-excluded URLs.
INFO:neuracrawl_tuner_lib:Saved excluded and non-excluded URLs.


## Interesting URLs (used to find common content to exclude)

In [5]:
await extract_interesting_urls("")

INFO:neuracrawl_tuner_lib:Extracting interesting URLs...
INFO:neuracrawl_tuner_lib:Found 53 non-excluded URLs.
INFO:neuracrawl_tuner_lib:Processing prompt (1/1)...
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:neuracrawl_tuner_lib:Processed prompt (1/1).
INFO:neuracrawl_tuner_lib:Summarizing 1 batches...
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:neuracrawl_tuner_lib:Summarized 1 batches and found 12 interesting URLs.
INFO:neuracrawl_tuner_lib:Extracted 12 interesting URLs.


In [ ]:
await download_interesting_urls()

## CSS Selectors

In [ ]:
await extract_css_selectors(
)

[] viewer should use save_state.json, maybe only work with save_state.json instead of all the other files? then have a result object that gets plugged into the save_state.json that can include extra data?
- better method splitting, service classes, more models for example for counts of file extensions, frequent urls
- one file with all kept urls and one with all not kept urls
- say with what version we are working when starting sth, like v_000 (latest) or so
- auto ai feedback
- links are getting bad?
- maybe make feedback tab field editable?
- <a aria-label="Online-Dienst 'Bewohnerparkausweis beantragen / verwalten' starten" class="linklist-boxed__link" href="#" onclick="window.open('https://eservice.siegburg.de/bewohnerparken');return false;">
- remove img and figures
- error analysis like look at page xyz, why is this text at the bottom missing, ai gets the text, css selectors and html and markdown and will figure out the exact reason
- sitemap deduplizieren mit normalisierung
- rendered view of raw html for seeing if it was even in the originial html content of raw html
- for downloading the pages juse crawl 4 ai instead of the normal httpx WITH SAME SCROLLING ETC SETTINGS AS IN NEURACRAWL
- split up css selector extraction and application to be able to change underlaying interesting urls easily without purging the selectors
- exclude by css selector on website
- show website urls in the select of xcode app, also at bottom left, not only folder name

# Manual CSS Selector Application

In [ ]:
apply_css_selectors()